#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.callback import callback, callback_function
from arena_api.system import system

#### Callbacks: Polling
>    This example demonstrates configuring a callback with polling. Polling
    allows for callbacks to be invoked over time.

In [ ]:
"""
This function waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


##### Must have the decorator on the callback function
> node.on_update requires node as its first parameter<br>
This function is triggered when the callback event is triggered

In [ ]:
@callback_function.node.on_update
def print_node_value(node):
    print(f"\tTemperature: {node.value}")

##### Register the callback on DeviceTemperature node
>  Nodes are polled through their node maps. This example demonstrates
    polling the device temperature node. It has a polling time of 1 second,
    which means that its callback will not be invoked within 1 second of
    the last time it has been polled.

In [ ]:
print("Registering callback on DeviceTemperature node")

nodemap = device.nodemap
node = nodemap.get_node("DeviceTemperature")

handle = callback.register(node, print_node_value)

##### Poll at regular intervals
> - The callback will only be invoked if the cumulative elapsed time since
    the last callback is larger than the polling time.
> - We calculate the elapsed
    time since the last callback, and pass it to the function maually, in miliseconds.

In [ ]:
curr_time = time.time()
time_of_last_call = time.time()

print("Polling at regular intervals:")
for i in range(6):
    time.sleep(0.5)
    curr_time = time.time()
    print("\tPolled")
    nodemap.poll(int(1000 * curr_time - 1000 * time_of_last_call))
    time_of_last_call = curr_time


##### Deregister each handle in the handle list
> Must be called before device is destroyed

In [ ]:
print("Deregistering callback")
callback.deregister(handle)

system.destroy_device()
print("Destroyed device(s)")